# RLLM SDK Quick Start: Make Any Agent Trainable With Almost No Adaptation

This tutorial shows how to make **any existing agent code** trainable with minimal changes.

**The key insight:** Just replace your OpenAI client with the SDK client, and everything is automatically tracked for training!

*Note: This tutorial focuses on showing you the mechanics. We'll explain how training works at the end.*

## Step 1: Start the Proxy

Start the proxy for testing. During training, the Trainer manages this automatically. The proxy logs all LLM calls to a SQLite database.

In [8]:
from pathlib import Path
from rllm.sdk.proxy.proxy_manager import ProxyManager

# Setup
DB_PATH = "/tmp/rllm_demo.db"
MODEL = "gpt-4o-mini"

openai_api_key = "sk-xxx"  # Fill your openai api key

# Clean up
Path(DB_PATH).unlink(missing_ok=True)

# Start proxy
proxy_manager = ProxyManager(proxy_port=4000, admin_token="my-shared-secret")
config = {
    "model_list": [
        {
            "model_name": MODEL,
            "litellm_params": {
                "model": MODEL,
                "api_key": openai_api_key,
            },
        }
    ]
}
proxy_manager.start_proxy_subprocess(config=config, db_path=DB_PATH, project="demo")
proxy_url = proxy_manager.get_proxy_url(include_v1=True)

print(f"✓ Proxy started at {proxy_url}")
print(f"✓ Database: {DB_PATH}")

✓ Proxy started at http://127.0.0.1:4000/v1
✓ Database: /tmp/rllm_demo.db


In [2]:
!python rllm/examples/solver_judge/prepare_countdown_data.py

README.md: 100%|███████████████████████████████| 314/314 [00:00<00:00, 3.51MB/s]
data/train-00000-of-00001.parquet: 100%|███| 2.85M/2.85M [00:00<00:00, 4.94MB/s]
Generating train split: 100%|█| 490364/490364 [00:00<00:00, 4867126.43 examples/
Map: 100%|████████████████████████| 1024/1024 [00:00<00:00, 21976.67 examples/s]
Train dataset size: 489340
Test dataset size: 1024
Stage 2 train dataset size: 50000
Stage 3 train dataset size: 50000
Train dataset path: /workspace/rllm/rllm/data/datasets/countdown/train.parquet
Test dataset path: /workspace/rllm/rllm/data/datasets/countdown/test.parquet
Stage 2 train dataset path: /workspace/rllm/rllm/data/datasets/countdown/stage2_train.parquet
Stage 3 train dataset path: /workspace/rllm/rllm/data/datasets/countdown/stage3_train.parquet

Sample train example:
{'target': 78, 'nums': array([30, 32, 76]), 'question': 'Using the numbers 30, 32, 76, find a way to reach the target number 78. You can use basic arithmetic operations (+, -, *, /) and each

In [3]:
from rllm.data.dataset import DatasetRegistry

train_dataset = DatasetRegistry.load_dataset("countdown", "train")
test_dataset = DatasetRegistry.load_dataset("countdown", "test")

train_dataset[0]

{'target': 78,
 'nums': [30, 32, 76],
 'question': 'Using the numbers 30, 32, 76, find a way to reach the target number 78. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your step-by-step calculation and output the final answer within <answer>...</answer>, for example <answer> (1 + 2) / 3 </answer>.',
 'ground_truth': '78',
 'data_source': 'countdown'}

**The Countdown Task:**  
Given a set of numbers and a target, find an arithmetic expression using those numbers to reach the target. Each number can be used at most once. For example: numbers `[30, 32, 76]` and target `78` → solution could be `76 + 32 - 30 = 78`.

## Step 2: Your Original Agent Code

A typical agent using the standard OpenAI client. This agent follows a Solver-Judge workflow: generate multiple solution attempts, then select the best one.

In [9]:
from openai import AsyncOpenAI
import re

judge_prompt = """You are an expert verifier. Given a countdown problem and multiple solution attempts, select a correct solution.
Problem:
{problem}
Solutions to evaluate:
{solutions}
A correct solution must satisfy the following criteria:
1. The solution uses only the given numbers.
2. Each number is used exactly once.
3. Only basic arithmetic operations (+, -, *, /) are used.
4. The calculation results in the target number.
5. The final answer is clearly marked within <answer>...</answer> tags.
Output the index of your selected solution within <answer>...</answer> tags, e.g., <answer>1</answer> for the first solution, <answer>2</answer> for the second solution, etc. If multiple solutions are correct, output the index of the first correct solution."""


class CountdownAgent:
    """A simple math solving agent - ORIGINAL VERSION"""

    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # Standard OpenAI client
        self.client = AsyncOpenAI(api_key=api_key)
        self.model = model

    async def solve(self, problem: str) -> str:
        """Solve a math problem."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{problem}. Output the final answer within <answer>...</answer>"}],
            max_tokens=1000,
        )
        return self.parse_solver_answer(response.choices[0].message.content)

    async def judge(self, problem, solutions: list[str]) -> str:
        """Judge multiple solutions to a problem."""
        formatted_solutions = "\n".join([f"Solution {i + 1}:\n{sol}\n" for i, sol in enumerate(solutions)])
        prompt = judge_prompt.format(problem=problem, solutions=formatted_solutions)

        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000,
        )
        return response.choices[0].message.content

    def parse_solver_answer(self, solution):
        # Find all <answer> tags and return the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", solution, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            return "<answer>" + answer_matches[-1].strip() + "</answer>"
        return "No solution found"

    def parse_selected_solution(self, judgment, solutions):
        # Find all <answer> tags and use the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", judgment, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            answer_text = answer_matches[-1].strip()
            try:
                solution_index = int(answer_text)
                return solutions[solution_index - 1]
            except (ValueError, IndexError):
                return ""
        return ""

    async def run(self, problem: str, n_solutions: int = 2) -> str:
        """Generate multiple solutions and judge them."""
        solutions = []
        for i in range(n_solutions):
            sol = await self.solve(problem)
            solutions.append(sol)

        judgment = await self.judge(problem, solutions)
        selected_solution = self.parse_selected_solution(judgment, solutions)
        return selected_solution


# Use it
agent = CountdownAgent(api_key=openai_api_key, model=MODEL)
result = await agent.run(train_dataset[0]["question"])

print(result)

<answer>(76 + 32) - 30</answer>


## Step 3: Make It Trainable (2 Simple Changes!)

**Change 1:** Import the SDK client instead of OpenAI client  
**Change 2:** Point to the proxy URL

**What's `session()`?** A lightweight primitive that tracks all LLM calls within its scope and injects metadata into each call. Everything inside `with session()` is automatically grouped and retrievable via `sess._uid`.

In [10]:
from rllm.sdk import get_chat_client_async, session  # Change 1: Import SDK client
import re


class TrainableAgent:
    """A simple math solving agent - TRAINABLE VERSION"""

    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # Replace standard OpenAI client with SDK client
        # self.client = AsyncOpenAI(api_key=api_key)
        self.client = get_chat_client_async(api_key=api_key, base_url=proxy_url)
        self.model = model

    async def solve(self, problem: str) -> str:
        """Solve a math problem."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{problem}. Output the final answer within <answer>...</answer>"}],
            max_tokens=1000,
        )
        return self.parse_solver_answer(response.choices[0].message.content)

    async def judge(self, problem, solutions: list[str]) -> str:
        """Judge multiple solutions to a problem."""
        formatted_solutions = "\n".join([f"Solution {i + 1}:\n{sol}\n" for i, sol in enumerate(solutions)])
        prompt = judge_prompt.format(problem=problem, solutions=formatted_solutions)

        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000,
        )
        return self.parse_solver_answer(response.choices[0].message.content)

    def parse_solver_answer(self, solution):
        # Find all <answer> tags and return the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", solution, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            return "<answer>" + answer_matches[-1].strip() + "</answer>"
        return "No solution found"

    def parse_selected_solution(self, judgment, solutions):
        # Find all <answer> tags and use the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", judgment, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            answer_text = answer_matches[-1].strip()
            try:
                solution_index = int(answer_text)
                return solutions[solution_index - 1]
            except (ValueError, IndexError):
                return ""
        return ""

    async def run(self, problem: str, n_solutions: int = 2) -> str:
        """Generate multiple solutions and judge them."""
        solutions = []
        for _ in range(n_solutions):
            sol = await self.solve(problem)
            solutions.append(sol)

        judgment = await self.judge(problem, solutions)
        selected_solution = self.parse_selected_solution(judgment, solutions)
        return selected_solution


# # Use it
agent = TrainableAgent(api_key=openai_api_key, model=MODEL)
with session() as sess:
    result = await agent.run(train_dataset[0]["question"])

print(result)

<answer>(76 - 30) + 32</answer>


### Why This Makes It Trainable: Automatic LLM Call Tracking

The `session()` primitive enables training by capturing every LLM interaction. You can access all traces directly via `sess.llm_calls`:

Each trace contains:
- `input`: Prompt messages sent to the model
- `output`: Model's response
- `tokens`: Exact token IDs (ensures correctness, bypasses retokenization issues)

In [11]:
# Access traces directly from the session
traces = sess.llm_calls

print(f"✅ Retrieved {len(traces)} trace(s)\n")

# Inspect the first trace
trace = traces[0]

print("=" * 70)
print("TRACE DETAILS")
print("=" * 70)
print(f"Model: {trace.model}")
print(f"\nInput Messages:")
for msg in trace.input["messages"]:
    print(f"  [{msg['role']}]: {msg['content']}")
print(f"\nOutput:")
print(f"  {trace.output['choices'][0]['message']['content']}")
print("=" * 70)

print("\n💡 This trace contains everything you need for training!")

✅ Retrieved 3 trace(s)

TRACE DETAILS
Model: gpt-4o-mini

Input Messages:
  [user]: Using the numbers 30, 32, 76, find a way to reach the target number 78. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your step-by-step calculation and output the final answer within <answer>...</answer>, for example <answer> (1 + 2) / 3 </answer>.. Output the final answer within <answer>...</answer>

Output:
  To reach the target number 78 using the numbers 30, 32, and 76 with basic arithmetic operations, we can follow these steps:

1. **First, let’s add 30 to 76**:
   \[
   30 + 76 = 106
   \]

2. **Now, since we want to reach 78, we can subtract 32 from the result obtained in step 1**:
   \[
   106 - 32 = 74
   \]

3. **Next, we need two more to reach 78, which we can achieve by adding 4**:
   Since we are looking for a series of operations, we realize that directly with these numbers, we can try instead:

   Another way could be:
   - **Subtract 30 

## Step 4: Add Rewards and Train

Define a reward function that scores agent outputs, then pass it to the trainer:

In [11]:
train_dataset[0]

{'target': 78,
 'nums': [30, 32, 76],
 'question': 'Using the numbers 30, 32, 76, find a way to reach the target number 78. You can use basic arithmetic operations (+, -, *, /) and each number can only be used once. Show your step-by-step calculation and output the final answer within <answer>...</answer>, for example <answer> (1 + 2) / 3 </answer>.',
 'ground_truth': '78',
 'data_source': 'countdown'}

In [12]:
def extract_solution(solution_str):
    # Look for answer pattern in the entire string
    answer_pattern = r"<answer>(.*?)</answer>"
    match = re.finditer(answer_pattern, solution_str)
    matches = list(match)
    if matches:
        final_answer = matches[-1].group(1).strip()
    else:
        final_answer = None
    return final_answer


def validate_equation(equation_str, available_numbers):
    """Validate that equation only uses available numbers and each number once."""
    try:
        # Extract all numbers from the equation
        numbers_in_eq = [int(n) for n in re.findall(r"\d+", equation_str)]

        # Check if all numbers in equation are available
        available_numbers = sorted(available_numbers)
        numbers_in_eq = sorted(numbers_in_eq)

        # Each number should be used exactly once
        return numbers_in_eq == available_numbers
    except Exception:
        return False


def evaluate_equation(equation_str):
    """Safely evaluate the arithmetic equation using eval() with precautions."""
    try:
        # Define a regex pattern that only allows numbers, operators, parentheses, and whitespace
        allowed_pattern = r"^[\d+\-*/().\s]+$"
        if not re.match(allowed_pattern, equation_str):
            raise ValueError("Invalid characters in equation.")

        # Evaluate the equation with restricted globals and locals
        result = eval(equation_str, {"__builtins__": None}, {})
        return result
    except Exception:
        return None


def reward_fn(solution_str, numbers, target):
    """The scoring function for countdown task.

    Args:
        solution_str: the solution text
        numbers: list of numbers
        target: target number

    Returns:
        float: 1.0 if correct, 0.0 if incorrectet
    """
    equation = extract_solution(solution_str=solution_str)

    if equation is None:
        return 0.0

    # Validate equation uses correct numbers
    if not validate_equation(equation, numbers):
        return 0.0

    # Evaluate equation
    try:
        result = evaluate_equation(equation)

        if result is None:
            return 0.0

        if abs(result - target) < 1e-5:  # Account for floating point precision
            return 1.0
        else:
            return 0.0
    except Exception:
        return 0.0


async def rollout_v1(question: str, ground_truth: str, nums: list, target: float, model="Qwen/Qwen3-4B-Instruct-2507", **kwargs) -> float:
    # we need to provide an rollout function that return a reward
    agent = TrainableAgent(api_key=openai_api_key, model=model)
    # agent = TrainableAgent(api_key=openai_api_key, model="gpt-4o-mini")
    response = await agent.run(question)
    print(response)
    reward = reward_fn(response, nums, target)
    return reward


await rollout_v1(**train_dataset[0], model="gpt-4o-mini")

<answer>(76 + 32) - 30</answer>


1.0

In [13]:
# Training
from rllm.trainer import AgentTrainer
from hydra import initialize_config_dir, compose
import os

with initialize_config_dir(config_dir="/workspace/rllm/examples/sdk", version_base=None):
    config = compose(config_name="tutorial_config")

trainer = AgentTrainer(
    agent_run_func=rollout_v1,  # or use rollout_v2 for step-level rewards
    config=config,
    train_dataset=train_dataset,
    val_dataset=test_dataset,
)

In [ ]:
trainer.train()

2025-11-19 03:52:30,870	INFO worker.py:1908 -- Started a local Ray instance. View the dashboard at 127.0.0.1:8265 
2025-11-19 03:52:31,698	INFO packaging.py:588 -- Creating a file package for local module '/workspace'.
2025-11-19 03:52:32,809	INFO packaging.py:380 -- Pushing file package 'gcs://_ray_pkg_d80ef6786f7cd5fa.zip' (50.50MiB) to Ray cluster...
2025-11-19 03:52:32,998	INFO packaging.py:393 -- Successfully pushed file package 'gcs://_ray_pkg_d80ef6786f7cd5fa.zip'.


(TaskRunner pid=4769) TaskRunner hostname: modal, PID: 4769
(TaskRunner pid=4769) {'actor_rollout_ref': {'actor': {'_target_': 'verl.workers.config.FSDPActorConfig',
(TaskRunner pid=4769)                                  'checkpoint': {'_target_': 'verl.trainer.config.CheckpointConfig',
(TaskRunner pid=4769)                                                 'async_save': False,
(TaskRunner pid=4769)                                                 'load_contents': ['model',
(TaskRunner pid=4769)                                                                   'optimizer',
(TaskRunner pid=4769)                                                                   'extra'],
(TaskRunner pid=4769)                                                 'save_contents': ['model',
(TaskRunner pid=4769)                                                                   'optimizer',
(TaskRunner pid=4769)                                                                   'extra']},
(TaskRunner pid=4769)       

(TaskRunner pid=4769) /workspace/rllm/verl/verl/trainer/ppo/ray_trainer.py:326: UserWarning: Disabled critic as algorithm.adv_estimator != gae. If it is not intended, please set critic.enable=True
(TaskRunner pid=4769)   self.use_critic = need_critic(self.config)
Generating train split: 0 examples [00:00, ? examples/s]


(TaskRunner pid=4769) dataset len: 489340
(TaskRunner pid=4769) Using dataset class: RLHFDataset
(TaskRunner pid=4769) dataset len: 1024
(TaskRunner pid=4769) Size of train dataloader: 122335, Size of val dataloader: 1
(TaskRunner pid=4769) Total training steps: 122335


Generating train split: 489340 examples [00:00, 1193037.41 examples/s]
Generating train split: 1024 examples [00:00, 260632.76 examples/s]


...

Generating trajectories:  69%|█████████████████████████████████▋               | 11/16 [00:13<00:03,  1.28it/s]


(TaskRunner pid=4769) <answer>92 - 43 - 12 - 9</answer>
(TaskRunner pid=4769) [e23b4d33-99ce-4edb-86ca-e08daa6c4edf:2:1] Rollout completed with reward: 1.0


Generating trajectories:  75%|████████████████████████████████████▊            | 12/16 [00:14<00:02,  1.35it/s]


(TaskRunner pid=4769) 
(TaskRunner pid=4769) [49b5d3d2-07da-448a-87db-102cd147ccf9:3:1] Rollout completed with reward: 0.0
(TaskRunner pid=4769) 
(TaskRunner pid=4769) [49b5d3d2-07da-448a-87db-102cd147ccf9:0:1] Rollout completed with reward: 0.0
(TaskRunner pid=4769) 
(TaskRunner pid=4769) [49b5d3d2-07da-448a-87db-102cd147ccf9:2:1] Rollout completed with reward: 0.0
(TaskRunner pid=4769) 
(TaskRunner pid=4769) [49b5d3d2-07da-448a-87db-102cd147ccf9:1:1] Rollout completed with reward: 0.0
(TaskRunner pid=4769) Total time for generating trajectories: 30.49 seconds


Generating trajectories: 100%|█████████████████████████████████████████████████| 16/16 [00:30<00:00,  1.91s/it]


## Bonus: Using @trajectory Decorator for Step-Level Control

The `@trajectory` decorator is **equivalent to `with session()`** - both track LLM calls using contextvar.

**Key difference:** Both provide `.steps` access for fine-grained control:
- `with session() as sess:` → `sess.steps` 
- `@trajectory(name="...")` → returns `TrajectoryView` with `.steps`

**When to use:**
- `with session()`: Simple episode tracking
- `@trajectory`: Multi-step agents where you want explicit step-level rewards (e.g., reward solver differently from judge)

In [ ]:
from rllm.sdk import trajectory
from rllm.sdk.protocol import TrajectoryView


class TrainableAgentV2:
    """A simple math solving agent - TRAINABLE VERSION V2"""

    def __init__(self, api_key: str, model: str = "gpt-4o-mini"):
        # Replace standard OpenAI client with SDK client
        # self.client = AsyncOpenAI(api_key=api_key)
        self.client = get_chat_client_async(api_key=api_key, base_url=proxy_url)
        self.model = model

    @trajectory(name="solver")
    async def solve(self, problem: str) -> str:
        """Solve a math problem."""
        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": f"{problem}. Output the final answer within <answer>...</answer>"}],
            max_tokens=1000,
        )
        return response.choices[0].message.content

    @trajectory(name="judge")
    async def judge(self, problem, solutions: list[str]) -> str:
        """Judge multiple solutions to a problem."""
        formatted_solutions = "\n".join([f"Solution {i + 1}:\n{sol}\n" for i, sol in enumerate(solutions)])
        prompt = judge_prompt.format(problem=problem, solutions=formatted_solutions)

        response = await self.client.chat.completions.create(
            model=self.model,
            messages=[{"role": "user", "content": prompt}],
            max_tokens=1000,
        )
        return self.parse_solver_answer(response.choices[0].message.content)

    def parse_solver_answer(self, solution):
        # Find all <answer> tags and return the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", solution, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            return "<answer>" + answer_matches[-1].strip() + "</answer>"
        return "No solution found"

    def parse_selected_solution(self, judgment, solutions):
        # Find all <answer> tags and use the last one
        answer_matches = re.findall(r"<answer>(.*?)</answer>", judgment, re.IGNORECASE | re.DOTALL)
        if answer_matches:
            answer_text = answer_matches[-1].strip()
            try:
                solution_index = int(answer_text)
                return solutions[solution_index - 1]
            except (ValueError, IndexError):
                return ""
        return ""

    async def run(self, problem: str, n_solutions: int = 2, ground_truth: str = None) -> str:
        """Generate multiple solutions and judge them."""
        solutions = []
        for _ in range(n_solutions):
            sol = await self.solve(problem)
            solutions.append(sol)

        judgment = await self.judge(problem, solutions)
        selected_solution = self.parse_selected_solution(judgment.result, solutions)

        # assign reward for each step in trajectory
        for sol in solutions:
            sol.reward = reward_fn(sol.result, ground_truth)
            sol.steps[0].reward = sol.reward

        judgment.reward = reward_fn(selected_solution, ground_truth)
        judgment.steps[0].reward = judgment.reward

        return solutions + [judgment]


# Use it
async def rollout_v2(question: str, ground_truth: str, **kwargs) -> list[TrajectoryView]:
    agent = TrainableAgentV2(None, model="Qwen/Qwen3-4B-Instruct-2507")
    trajs = await agent.run(question, ground_truth=ground_truth)
    return trajs

## How Does Training Work?

Here's what happens under the hood:

1. **Trace Collection:** The proxy captures all LLM calls (inputs, outputs, tokens, latency)
2. **Reward Assignment:** You define what's good (correct answer = 1.0, wrong = 0.0)
3. **Training Loop:** The trainer feeds traces + rewards to the model
4. **Learning:** The model adjusts weights to maximize rewards
5. **Improvement:** Over time, the model learns successful behaviors

This is reinforcement learning: try different approaches, get feedback, learn what works.

## Design Details (For The Curious)

**Why a proxy?**  
Transparent LLM call interception without modifying agent code. Works with any OpenAI-compatible API.

**How does session tracking work?**  
Uses Python's **contextvar** for automatic context propagation. `with session()` or `@trajectory` creates a context that automatically groups all LLM calls inside it. Thread-safe, zero manual tracking.

**Session vs Trajectory:**  
Both use contextvar under the hood:
- `with session()`: Returns session object with `._uid` for retrieval
- `@trajectory`: Returns `TrajectoryView` with `.steps` for fine-grained control

**Why SQLite storage?**  
Offline training with no live service dependencies. Query and analyze traces anytime.

In [ ]:
# Cleanup
proxy_manager.shutdown_proxy()
print("✓ Proxy shutdown complete")